In [275]:
#import dependencies
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm, tqdm_notebook
import pickle

In [233]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [234]:
df = pd.read_csv('../NCAA_data.csv')
df.iloc[:20]

,player_id,name,class,college,big_conf_flag,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Player,cincinnati,0,142,292,0.587,0.586,0.494,0.581,116,81,2042.000000,1279,124,0.222000,2.0,869,0.592,14.300000,139.000000,0.346000,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Player,indiana,1,403,52,0.541,0.455,0.249,0.790,128,73,3260.000000,2100,129,0.414000,283.0,429,0.572,13.600000,218.000000,0.145000,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Player,uconn,1,124,193,0.542,0.542,0.555,0.656,138,83,2184.000000,863,86,0.329162,0.0,880,0.573,23.000000,160.000000,0.237000,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,1,479,10,0.486,0.416,0.257,0.822,108,70,2149.000000,1650,186,0.355000,195.0,319,0.527,15.600000,188.000000,0.158000,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,university of louisiana at monroe,0,147,42,0.507,0.428,0.357,0.769,55,80,1938.000000,1062,90,0.318000,130.0,449,0.551,20.000000,241.000000,0.165000,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200
5,eddie-house-1,Eddie House,Player,arizona state,1,405,19,0.495,0.425,0.187,0.798,124,73,4136.000000,2044,258,0.366000,250.0,502,0.523,10.800000,195.000000,0.113000,3.916828,0.183752,19.767892,2.495164,2.417795,4.854932,1.885880
6,eduardo-najera-1,Eduardo Najera,Player,oklahoma,1,215,89,0.460,0.430,0.354,0.669,126,80,3001.000000,1646,193,0.288000,85.0,910,0.495,13.100000,201.000000,0.183000,2.865711,1.186271,21.939354,2.572476,1.132956,12.129290,2.679107
7,lavor-postell-1,Lavor Postell,Player,st. john's,1,205,58,0.470,0.430,0.383,0.789,129,77,2845.000000,1427,129,0.326000,92.0,716,0.526,13.800000,172.000000,0.115000,2.882250,0.815466,20.063269,1.813708,1.293497,10.066784,2.418278
8,hanno-mottola-1,Hanno Mottola,Bust,utah,1,139,50,0.533,0.500,0.415,0.774,120,83,2162.000000,1492,37,0.333000,71.0,551,0.580,14.900000,159.000000,0.207000,2.571693,0.925069,27.604070,0.684551,1.313599,10.194265,2.941721
9,chris-carrawell-1,Chris Carrawell,Bust,duke,1,309,114,0.514,0.486,0.415,0.667,136,78,2494.900238,1455,105,0.360000,62.0,608,0.545,15.219622,177.152482,0.181834,4.954106,1.827728,23.327586,1.683434,0.994028,9.747885,2.840234


In [235]:
df.columns

Index(['player_id', 'name', 'class', 'college', 'big_conf_flag', 'assists',
       'blocks', 'effective_field_goal_percentage', 'field_goal_percentage',
       'free_throw_attempt_rate', 'free_throw_percentage', 'games_played',
       'height', 'minutes_played', 'points', 'steals',
       'three_point_percentage', 'three_pointers', 'total_rebounds',
       'true_shooting_percentage', 'turnover_percentage', 'turnovers',
       'win_shares_per_40_minutes', 'assists_per_40', 'blocks_per_40',
       'points_per_40', 'steals_per_40', 'three_pointers_per_40',
       'total_rebounds_per_40', 'turnovers_per_40'],
      dtype='object')

In [236]:
y = df["class"]

In [237]:
from sklearn.preprocessing import LabelEncoder
y = df["class"]
labels = LabelEncoder()
labels.fit(y)
encoded_y = labels.transform(y)

In [238]:
en_y = encoded_y
df["code"]=en_y
df.head()

,player_id,name,class,college,big_conf_flag,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40,code
0,kenyon-martin-1,Kenyon Martin,Player,cincinnati,0,142,292,0.587,0.586,0.494,0.581,116,81,2042.0,1279,124,0.222000,2.0,869,0.592,14.3,139.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821,1
1,aj-guyton-1,A.J. Guyton,Player,indiana,1,403,52,0.541,0.455,0.249,0.790,128,73,3260.0,2100,129,0.414000,283.0,429,0.572,13.6,218.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847,1
2,jake-voskuhl-1,Jake Voskuhl,Player,uconn,1,124,193,0.542,0.542,0.555,0.656,138,83,2184.0,863,86,0.329162,0.0,880,0.573,23.0,160.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403,1
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,1,479,10,0.486,0.416,0.257,0.822,108,70,2149.0,1650,186,0.355000,195.0,319,0.527,15.6,188.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302,0
4,mike-smith-1,Mike Smith,Bust,university of louisiana at monroe,0,147,42,0.507,0.428,0.357,0.769,55,80,1938.0,1062,90,0.318000,130.0,449,0.551,20.0,241.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200,0


In [239]:
labels_df = pd.DataFrame({"class": y, "code": encoded_y}).drop_duplicates(subset=None, keep="first")
labels_df

,class,code
0,Player,1
3,Bust,0


In [240]:
X = df.drop(["code", "player_id", "name", "class", "college",  'big_conf_flag', 'assists',
       'blocks', 'minutes_played', 'points', 'steals', 'three_pointers', 'total_rebounds', 'turnovers',], axis=1)
feature_names = X.columns
X.head()

,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,three_point_percentage,true_shooting_percentage,turnover_percentage,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,0.587,0.586,0.494,0.581,116,81,0.222000,0.592,14.3,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,0.541,0.455,0.249,0.790,128,73,0.414000,0.572,13.6,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,0.542,0.542,0.555,0.656,138,83,0.329162,0.573,23.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,0.486,0.416,0.257,0.822,108,70,0.355000,0.527,15.6,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,0.507,0.428,0.357,0.769,55,80,0.318000,0.551,20.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [241]:
print(X.shape)
print(y.shape)

(853, 17)
(853,)


In [242]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=42)

In [243]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
# y_scaler = MinMaxScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)


In [244]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.6635514018691588

In [245]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.6635514018691588


In [199]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.05454408, 0.07384617, 0.05096299, 0.04622933, 0.1145453 ,
       0.02759095, 0.04684866, 0.05995488, 0.05131917, 0.07376494,
       0.05144778, 0.05733649, 0.0588658 , 0.06065353, 0.04746257,
       0.05005979, 0.07456755])

In [246]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.11398710528339111, 'games_played'),
 (0.07579959373322935, 'turnovers_per_40'),
 (0.07447107415489745, 'win_shares_per_40_minutes'),
 (0.07162199322698666, 'field_goal_percentage'),
 (0.062419543941642654, 'blocks_per_40'),
 (0.06131315393936584, 'points_per_40'),
 (0.06017842876901885, 'steals_per_40'),
 (0.05604505978106013, 'effective_field_goal_percentage'),
 (0.054256889329527797, 'total_rebounds_per_40'),
 (0.05349615695412456, 'true_shooting_percentage'),
 (0.05140745391557851, 'free_throw_attempt_rate'),
 (0.04882363395055714, 'three_pointers_per_40'),
 (0.04835270233708412, 'turnover_percentage'),
 (0.04789171441411229, 'three_point_percentage'),
 (0.04688899745392502, 'assists_per_40'),
 (0.04527014948674774, 'free_throw_percentage'),
 (0.027776349328750782, 'height')]

In [247]:
from sklearn.metrics import classification_report
predictions = rf.predict(X_test_scaled)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.53      0.11      0.18        73
           1       0.67      0.95      0.79       141

    accuracy                           0.66       214
   macro avg       0.60      0.53      0.49       214
weighted avg       0.63      0.66      0.58       214



In [17]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

In [37]:
# Train the model with GridSearch
# Create the parameter grid based on the results of random search 
param_grid = { 'max_depth': [10,20,30,50,100],
    'n_estimators': [100, 200, 300, 500]
}
# Create a based model
model2 = RandomForestClassifier()
# Instantiate the grid search model
grid = GridSearchCV(model2, param_grid, verbose=3)

In [38]:
grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_depth=10, n_estimators=100 ..................................
[CV] ...... max_depth=10, n_estimators=100, score=0.738, total=   0.1s
[CV] max_depth=10, n_estimators=100 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...... max_depth=10, n_estimators=100, score=0.761, total=   0.1s
[CV] max_depth=10, n_estimators=100 ..................................
[CV] ...... max_depth=10, n_estimators=100, score=0.755, total=   0.1s
[CV] max_depth=10, n_estimators=200 ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ...... max_depth=10, n_estimators=200, score=0.752, total=   0.2s
[CV] max_depth=10, n_estimators=200 ..................................
[CV] ...... max_depth=10, n_estimators=200, score=0.765, total=   0.2s
[CV] max_depth=10, n_estimators=200 ..................................
[CV] ...... max_depth=10, n_estimators=200, score=0.731, total=   0.2s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.757, total=   0.3s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.761, total=   0.3s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.736, total=   0.3s
[CV] max_depth=10, n_estimators=500 ..................................
[CV] ...... max_depth=10, n_estimators=500, score=0.738, total=   0.6s
[CV] max_depth=10, n_estimators=500 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   19.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [39]:
#check best params and scores
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 100, 'n_estimators': 300}
0.7605633802816901


In [41]:
#define model using best hyperparameters
model_opt =  RandomForestClassifier(max_depth = 100, n_estimators = 300)
model_opt.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
print(f"Training Data Score: {model_opt.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model_opt.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.6728971962616822


In [231]:
predictions2 = model_opt.predict(X_test_scaled)
print(classification_report(y_test, predictions2))

ValueError: Number of features of the model must match the input. Model n_features is 6 and input n_features is 17 

In [248]:
from sklearn.feature_selection import SelectFromModel
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")

In [249]:
X2 = df.drop(['player_id', 'name', 'class', 'college', 'assists', 'blocks',
    'field_goal_percentage', 'minutes_played', 'points', 'steals',
       'three_pointers', 'total_rebounds', 'turnovers', 'code', ], axis=1)
feature_names = X2.columns
X2.head()

,big_conf_flag,effective_field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,three_point_percentage,true_shooting_percentage,turnover_percentage,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,0,0.587,0.494,0.581,116,81,0.222000,0.592,14.3,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,1,0.541,0.249,0.790,128,73,0.414000,0.572,13.6,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,1,0.542,0.555,0.656,138,83,0.329162,0.573,23.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,1,0.486,0.257,0.822,108,70,0.355000,0.527,15.6,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,0,0.507,0.357,0.769,55,80,0.318000,0.551,20.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [250]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, encoded_y, random_state=42)

In [251]:
X2_scaler = MinMaxScaler().fit(X2_train)


X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)


In [252]:
select.fit(X2_train_scaled, y2_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,
                                                 random_state=42, verbose=0,
    

In [253]:
X2_train_scaled_l1 = select.transform(X2_train_scaled)

In [254]:
print("X2_train.shape:{}".format(X2_train_scaled.shape))

X2_train.shape:(639, 17)


In [255]:
print("X2_train_l1.shape:{}".format(X2_train_scaled_l1.shape))

X2_train_l1.shape:(639, 9)


In [256]:
mask = select.get_support()
print(mask)

[False  True False False  True False False  True False  True False  True
  True  True False  True  True]


In [257]:
print(sorted(zip(mask, feature_names), reverse=True))

[(True, 'win_shares_per_40_minutes'), (True, 'turnovers_per_40'), (True, 'true_shooting_percentage'), (True, 'total_rebounds_per_40'), (True, 'steals_per_40'), (True, 'points_per_40'), (True, 'games_played'), (True, 'effective_field_goal_percentage'), (True, 'blocks_per_40'), (False, 'turnover_percentage'), (False, 'three_pointers_per_40'), (False, 'three_point_percentage'), (False, 'height'), (False, 'free_throw_percentage'), (False, 'free_throw_attempt_rate'), (False, 'big_conf_flag'), (False, 'assists_per_40')]


In [287]:
df_select = df[['games_played','win_shares_per_40_minutes', 'turnovers_per_40', 'true_shooting_percentage', 'total_rebounds_per_40',
                'assists_per_40','steals_per_40','points_per_40','effective_field_goal_percentage','blocks_per_40', ]]

In [288]:
X3 = df_select

In [298]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, encoded_y, random_state=42)
X3_scaler = MinMaxScaler().fit(X3_train)
X3_train_scaled = X3_scaler.transform(X3_train)
X3_test_scaled = X3_scaler.transform(X3_test)

In [299]:
rf3 = RandomForestClassifier()
rf3 = rf3.fit(X3_train_scaled, y3_train)
rf3.score(X3_test_scaled, y3_test)

0.6869158878504673

In [300]:
print(f"Training Data Score: {rf3.score(X3_train_scaled, y3_train)}")
print(f"Testing Data Score: {rf3.score(X3_test_scaled, y3_test)}")

Training Data Score: 0.9890453834115805
Testing Data Score: 0.6869158878504673


In [301]:
feature3_names = X3.columns

In [302]:
importances3 = rf3.feature_importances_
sorted(zip(importances3, feature3_names), reverse=True)

[(0.14190338169651984, 'games_played'),
 (0.12152144569702736, 'win_shares_per_40_minutes'),
 (0.11195649286796125, 'total_rebounds_per_40'),
 (0.11096775741029777, 'turnovers_per_40'),
 (0.10498821096538732, 'effective_field_goal_percentage'),
 (0.09493725265691821, 'true_shooting_percentage'),
 (0.09396537975285867, 'steals_per_40'),
 (0.08466800489358176, 'points_per_40'),
 (0.07698374313749193, 'blocks_per_40'),
 (0.05810833092195592, 'assists_per_40')]

In [269]:
# Train the model with GridSearch
# Create the parameter grid based on the results of random search 
param_grid = { 'max_depth': [10,50,100,200, 300],
    'n_estimators': [100, 200, 300, 500, 1000]
}
# Create a based model
r4 = RandomForestClassifier()
# Instantiate the grid search model
grid = GridSearchCV(r4, param_grid, verbose=3)

In [270]:
grid.fit(X3_train_scaled, y3_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] max_depth=10, n_estimators=100 ..................................
[CV] ...... max_depth=10, n_estimators=100, score=0.766, total=   0.4s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] max_depth=10, n_estimators=100 ..................................
[CV] ...... max_depth=10, n_estimators=100, score=0.718, total=   0.4s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV] max_depth=10, n_estimators=100 ..................................
[CV] ...... max_depth=10, n_estimators=100, score=0.745, total=   0.4s
[CV] max_depth=10, n_estimators=200 ..................................
[CV] ...... max_depth=10, n_estimators=200, score=0.776, total=   0.7s
[CV] max_depth=10, n_estimators=200 ..................................
[CV] ...... max_depth=10, n_estimators=200, score=0.714, total=   0.7s
[CV] max_depth=10, n_estimators=200 ..................................
[CV] ...... max_depth=10, n_estimators=200, score=0.741, total=   0.8s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.762, total=   1.1s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.709, total=   1.1s
[CV] max_depth=10, n_estimators=300 ..................................
[CV] ...... max_depth=10, n_estimators=300, score=0.750, total=   1.2s
[CV] m

[CV] .... max_depth=200, n_estimators=1000, score=0.764, total=   3.7s
[CV] max_depth=300, n_estimators=100 .................................
[CV] ..... max_depth=300, n_estimators=100, score=0.752, total=   0.4s
[CV] max_depth=300, n_estimators=100 .................................
[CV] ..... max_depth=300, n_estimators=100, score=0.714, total=   0.4s
[CV] max_depth=300, n_estimators=100 .................................
[CV] ..... max_depth=300, n_estimators=100, score=0.726, total=   0.4s
[CV] max_depth=300, n_estimators=200 .................................
[CV] ..... max_depth=300, n_estimators=200, score=0.757, total=   0.9s
[CV] max_depth=300, n_estimators=200 .................................
[CV] ..... max_depth=300, n_estimators=200, score=0.718, total=   0.8s
[CV] max_depth=300, n_estimators=200 .................................
[CV] ..... max_depth=300, n_estimators=200, score=0.750, total=   0.8s
[CV] max_depth=300, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  2.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [226]:
#check best params and scores
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 10, 'n_estimators': 200}
0.7464788732394366


In [271]:
model3_opt =  RandomForestClassifier(max_depth = 10, n_estimators = 200)

In [303]:
model3_opt.fit(X3_train_scaled, y3_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [273]:
print(f"Training Data Score: {model3_opt.score(X3_train_scaled, y3_train)}")
print(f"Testing Data Score: {model3_opt.score(X3_test_scaled, y3_test)}")

Training Data Score: 0.9687010954616588
Testing Data Score: 0.6682242990654206


In [304]:
predictions3 = model3_opt.predict(X3_test_scaled)
print(classification_report(y3_test, predictions3))

              precision    recall  f1-score   support

           0       0.53      0.12      0.20        73
           1       0.68      0.94      0.79       141

    accuracy                           0.66       214
   macro avg       0.60      0.53      0.49       214
weighted avg       0.63      0.66      0.59       214



In [308]:
pickle.dump(model3_opt, open('model.pkl','wb'))

model = pickle.load(open('model.pkl','rb'))
print(model.predict([[116, .346, 2.7, 0.59, 17, 2.7, 2.4, 25, .587, 5.7]]))

[0]


In [277]:
print(model.predict([[4, 300, 500]]))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

X3

In [305]:
X3

,games_played,win_shares_per_40_minutes,turnovers_per_40,true_shooting_percentage,total_rebounds_per_40,assists_per_40,steals_per_40,points_per_40,effective_field_goal_percentage,blocks_per_40
0,116,0.346000,2.722821,0.592,17.022527,2.781587,2.428991,25.053869,0.587,5.719882
1,128,0.145000,2.674847,0.572,5.263804,4.944785,1.582822,25.766871,0.541,0.638037
2,138,0.237000,2.930403,0.573,16.117216,2.271062,1.575092,15.805861,0.542,3.534799
3,108,0.158000,3.499302,0.527,5.937645,8.915775,3.462075,30.711959,0.486,0.186133
4,55,0.165000,4.974200,0.551,9.267286,3.034056,1.857585,21.919505,0.507,0.866873
5,124,0.113000,1.885880,0.523,4.854932,3.916828,2.495164,19.767892,0.495,0.183752
6,126,0.183000,2.679107,0.495,12.129290,2.865711,2.572476,21.939354,0.460,1.186271
7,129,0.115000,2.418278,0.526,10.066784,2.882250,1.813708,20.063269,0.470,0.815466
8,120,0.207000,2.941721,0.580,10.194265,2.571693,0.684551,27.604070,0.533,0.925069
9,136,0.181834,2.840234,0.545,9.747885,4.954106,1.683434,23.327586,0.514,1.827728
